# Pretrain Bert on MLM and NSP Simultaneously

Ref:

(i) https://stackoverflow.com/questions/70122842/bert-pre-training-mlm-nsp?rq=1


(ii) https://stackoverflow.com/questions/65646925/how-to-train-bert-from-scratch-on-a-new-domain-for-both-mlm-and-nsp


(iii) https://www.thepythoncode.com/article/pretraining-bert-huggingface-transformers-in-python

In [ ]:
!pip install datasets transformers

     |████████████████████████████████| 298 kB 4.3 MB/s 
     |████████████████████████████████| 3.3 MB 42.6 MB/s 
     |████████████████████████████████| 61 kB 555 kB/s 
     |████████████████████████████████| 243 kB 57.7 MB/s 
     |████████████████████████████████| 132 kB 66.9 MB/s 
     |████████████████████████████████| 1.1 MB 58.5 MB/s 
     |████████████████████████████████| 3.3 MB 53.7 MB/s 
     |████████████████████████████████| 596 kB 50.6 MB/s 
     |████████████████████████████████| 895 kB 51.6 MB/s 
     |████████████████████████████████| 160 kB 60.9 MB/s 
     |████████████████████████████████| 271 kB 59.8 MB/s 
     |████████████████████████████████| 192 kB 53.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
from transformers import (
    BertTokenizer,
    BertTokenizerFast,
    BertConfig, 
    BertForPreTraining, 
    TextDatasetForNextSentencePrediction,
    DataCollatorForLanguageModeling,
    Trainer, 
    TrainingArguments
)
import torch
from datasets import load_dataset, concatenate_datasets

In [ ]:
# For sentence tokenization
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# CONFIGS

RANDOM_SEED=37

DATASET_LIMIT = 20_000

# MODEL_MAX_LEN = 256
NSP_DATESET_PATH = 'nsp.txt'
MLM_TRAIN_DATESET_PATH = 'mlm_train.txt'
MLM_TEST_DATESET_PATH = 'mlm_test.txt'
MLM_MASKING_PROB = .15

MODEL_NAME = "bert-base-uncased"
# MODEL_NAME = "bert-base-multilingual-uncased"

MODEL_SAVE_PATH = MODEL_NAME

## Load Dataset

In [ ]:
# wiki = load_dataset("wikipedia", "20200501.en", split="train")
# bookcorpus = load_dataset("bookcorpus", split="train")
# print(wiki.column_names, bookcorpus.column_names)
# # ['title', 'text'] ['text']

# wiki.remove_columns_("title")
# bert_dataset = concatenate_datasets([wiki, bookcorpus])


dataset = load_dataset("cc_news", split="train")

bert_dataset = dataset

Downloading:   0%|          | 0.00/1.75k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/932 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/845M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

Dataset cc_news downloaded and prepared to /root/.cache/huggingface/datasets/cc_news/plain_text/1.0.0/ae469e556251e6e7e20a789f93803c7de19d0c4311b6854ab072fecb4e401bd6. Subsequent calls will reuse this data.


In [ ]:
bert_dataset

Dataset({
    features: ['title', 'text', 'domain', 'date', 'description', 'url', 'image_url'],
    num_rows: 708241
})

In [ ]:
bert_dataset[0]

{'date': '2017-12-11 20:19:05',
 'description': "There's a surprising twist to Regina Willoughby's last season with\xa0Columbia City Ballet: It's also her 18-year-old daughter Melina's first season with the company.",
 'domain': 'www.pointemagazine.com',
 'image_url': 'https://pointe-img.rbl.ms/simage/https%3A%2F%2Fassets.rbl.ms%2F16807693%2F980x.png/2000%2C2000/3VnhNGWp75K4SwMx/img.png',
 'text': 'There\'s a surprising twist to Regina Willoughby\'s last season with Columbia City Ballet: It\'s also her 18-year-old daughter Melina\'s first season with the company. Regina, 40, will retire from the stage in March, just as her daughter starts her own career as a trainee. But for this one season, they\'re sharing the stage together.\nPerforming Side-By-Side In The Nutcracker\nRegina and Melina are not only dancing in the same Nutcracker this month, they\'re onstage at the same time: Regina is doing Snow Queen, while Melina is in the snow corps, and they\'re both in the Arabian divertissemen

### For MLM

In [ ]:
# def split_string(str, limit, sep=" "):
#     """
#     Split a long string into list of substrings each of
#     which has length less than the given limit.
#     """
#     words = str.split()
#     words = list(filter(lambda x: len(x)<limit, words))
#     if max(map(len, words)) > limit:
#         raise ValueError("limit is too small")
#     res, part, others = [], words[0], words[1:]
#     for word in others:
#         if len(sep)+len(word) > limit-len(part):
#             res.append(part)
#             part = word
#         else:
#             part += sep+word
#     if part:
#         res.append(part)
#     return res

In [ ]:
# def dataset_to_text(dataset, output_filename):
#   """
#   Write dataset to file
#   """
#   with open(output_filename, "w") as f:
#     for document in dataset["text"]:
#       if len(document)<=MODEL_MAX_LEN:
#         # If document is less than the max model length
#         print(document, file=f)
#       else:
#         splitted_substrings = split_string(document, MODEL_MAX_LEN)
#         for substring in splitted_substrings:
#           print(substring, file=f)

In [ ]:
# # split the dataset into training (90%) and testing (10%)
# d = bert_dataset.train_test_split(test_size=0.1, seed=RANDOM_SEED)

In [ ]:
# # save the training set to train.txt
# dataset_to_text(d["train"][:DATASET_LIMIT], MLM_TRAIN_DATESET_PATH)
# # save the testing set to test.txt
# dataset_to_text(d["test"][:DATASET_LIMIT//10], MLM_TEST_DATESET_PATH)

### For NSP
(1) One sentence per line. 

(2) Blank lines between documents


ref: https://stackoverflow.com/questions/65646925/how-to-train-bert-from-scratch-on-a-new-domain-for-both-mlm-and-nsp

In [ ]:
with open(NSP_DATESET_PATH, "w") as f:
  for document in bert_dataset[:DATASET_LIMIT]["text"]:
    # replace paragraph changes with fullstop for sentence segmentation
    document = document.replace('\n', '')
    for sentence in sent_tokenize(document):
      sentence = sentence.strip()
      if sentence != '':
          print(sentence, file=f)
    # line break for each document
    print('', file=f)

In [ ]:
# with open(NSP_DATESET_PATH, "w") as f:
#   for document in bert_dataset[:DATASET_LIMIT]["text"]:
#     # replace paragraph changes with fullstop for sentence segmentation
#     document = document.replace('\n', '')
#     for sentence in sent_tokenize(document):
#       sentence = sentence.strip()
#       if sentence != '':
#         if len(sentence)<=MODEL_MAX_LEN:
#           # If string is less than the max model length
#           print(sentence, file=f)
#         else:
#           splitted_substrings = split_string(sentence, MODEL_MAX_LEN)
#           for substring in splitted_substrings:
#             print(substring, file=f)
#     # line break for each document
#     print('', file=f)


# Remove final line breaks  
with open(NSP_DATESET_PATH) as f_input:
    data = f_input.read().rstrip('\n')

with open(NSP_DATESET_PATH, 'w') as f_output:    
    f_output.write(data)

In [ ]:
# Remove final line breaks
line_len = []  
with open(NSP_DATESET_PATH) as f_input:
    data = f_input.read()
    for line in data.split('\n'):
        line_len.append(len(line))

print(max(line_len))

10815


In [ ]:
lines = data.split('\n')

In [ ]:
lines[10]

'Joining Stafford are NYCB resident choreographer and soloist Justin Peck and ballet masters Craig Hall and Rebecca Krohn, both former dancers with the company.'

## Tokenizer

In [ ]:
bert_tokenizer = BertTokenizer.from_pretrained(MODEL_NAME, model_max_length=512)
# bert_tokenizer = BertTokenizerFast.from_pretrained(MODEL_NAME, max_len=512)

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc7043

In [ ]:
dir(bert_tokenizer)

['SPECIAL_TOKENS_ATTRIBUTES',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_tokens',
 '_additional_special_tokens',
 '_batch_encode_plus',
 '_batch_prepare_for_model',
 '_bos_token',
 '_cls_token',
 '_convert_id_to_token',
 '_convert_token_to_id',
 '_convert_token_to_id_with_added_voc',
 '_create_or_get_repo',
 '_create_trie',
 '_decode',
 '_decode_use_source_tokenizer',
 '_encode_plus',
 '_eos_token',
 '_eventual_warn_about_too_long_sequence',
 '_from_pretrained',
 '_get_padding_truncation_strategies',
 '_get_repo_url_from_name',
 '_mask_token',
 '_pad',
 '_pad_token',
 '_pad_token_type_id',
 '_push_to_hub',
 '_save

## Model

In [ ]:
config = BertConfig(
    num_hidden_layers=4, 
    num_attention_heads=4, 
)
model = BertForPreTraining(config)

In [ ]:
device = torch.device('cpu')# and move our model over to the selected device
model.to(device)

BertForPreTraining(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine

#### NSP

In [ ]:
dataset = TextDatasetForNextSentencePrediction(
    tokenizer=bert_tokenizer,
    file_path=NSP_DATESET_PATH,
    block_size = 256
)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:366: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,
Loading features from cached file cached_nsp_BertTokenizer_256_nsp.txt [took 15.486 s]


#### MLM


Use DataCollatorForLanguageModeling for masking and passing the labels that are generated from TextDatasetForNextSentencePrediction. DataCollatorForNextSentencePrediction has been removed, since it was doing the same thing with DataCollatorForLanguageModeling

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=bert_tokenizer, 
    mlm=True,
    mlm_probability= MLM_MASKING_PROB
)

### Training


In [ ]:
training_args = TrainingArguments(
    output_dir= "results",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size= 32,
    save_steps=1000,
    save_on_each_node=True,
    save_total_limit=2,
    prediction_loss_only=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
# import torch
# torch.cuda.empty_cache()

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

trainer.train()

***** Running training *****
  Num examples = 68283
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 10670


Step,Training Loss


RuntimeError: ignored

In [ ]:
trainer.save_model(MODEL_SAVE_PATH)